In [1]:
import warnings
import torch
import pandas as pd
import pytorch_lightning as pl

from botorch.sampling.samplers import SobolQMCNormalSampler
from sklearn.preprocessing import StandardScaler

from minerva.bayesopt import BayesianOptimisation 
from minerva.utils import get_index_from_lookup

warnings.filterwarnings('ignore')

## Notebook 5: Running Bayesian optimisation on training data 

This tutorial shows how to run an iteration of Bayesian optimisation given obtained experimental results. The experimental results in this notebook are experiments suggested from `sobol_initialisation.ipynb`. This notebook shows an example of how optimisation suggestions were obtained during the ML experimental campaign detailed in the manuscript.

In [2]:
tkwargs = {
        "dtype": torch.double,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    }

## 1. Loading training data 
- For instance, here we assume initial experimental training data was obtained from the initial experiments suggested in `sobol_initialisation.ipynb`
- We open this experimental data with the corresponding rxn_id's in a pandas dataframe

**Note that this same procedure is also applicable to when multiple batches of experiments have already been conducted** E.g. we would use all the conducted experiments as training data 

In [3]:
init_experiments_results = pd.read_csv('../experimental_campaigns/experiments/tutorial_data/initialisation_results.csv', index_col=0)
init_experiments_results.head(5)

,rxn_id,P_A%,Selectivity_A%
0,48464,0.0000,0.0000
1,48484,0.0000,0.0000
2,48586,0.0212,0.1607
3,48614,0.4183,0.8606
4,48650,0.2750,0.7504


In [4]:
# load the previously suggested experiments from 4_sobol_initialisation.ipynb
init_x_experiments = pd.read_csv('../experimental_campaigns/experiments/tutorial_data/initial_experiments.csv', index_col=0)
init_x_experiments.head(5)

,rxn_id,temperature,ligand,solvent,precursor,base,cosolvent,ligand_PC1,ligand_PC2,ligand_PC3,...,[Ni(oTol)(Cl)(PPh3)2],[Ni(oTol)(Cl)(TMEDA)],Cs2CO3,DBU,DIPEA,K3PO4,MeOH,none,water,temperature_descriptor
48464,48464,70,PPh3,MeTHF,[Ni(COD)(DQ)],Cs2CO3,MeOH,0.038384,0.281134,0.453264,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5
48484,48484,70,PPh3,MeCN,[Ni(oTol)(Cl)(TMEDA)],DBU,water,0.038384,0.281134,0.453264,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5
48586,48586,70,PPh3,iPrOH,[Ni(oTol)(Cl)(TMEDA)],K3PO4,water,0.038384,0.281134,0.453264,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5
48614,48614,70,PPh3,iPrOH,NiCl2.6H2O,DIPEA,MeOH,0.038384,0.281134,0.453264,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
48650,48650,70,PPh3,DEC,[Ni(COD)(DQ)],DIPEA,MeOH,0.038384,0.281134,0.453264,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5


In [5]:
# concatenate to create a training dataframe, appended to the last columns 
init_x_experiments['P_A%'] = init_experiments_results['P_A%'].to_list()
init_x_experiments['Selectivity_A%'] = init_experiments_results['Selectivity_A%'].to_list()
init_x_experiments.head(5)

,rxn_id,temperature,ligand,solvent,precursor,base,cosolvent,ligand_PC1,ligand_PC2,ligand_PC3,...,Cs2CO3,DBU,DIPEA,K3PO4,MeOH,none,water,temperature_descriptor,P_A%,Selectivity_A%
48464,48464,70,PPh3,MeTHF,[Ni(COD)(DQ)],Cs2CO3,MeOH,0.038384,0.281134,0.453264,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0000,0.0000
48484,48484,70,PPh3,MeCN,[Ni(oTol)(Cl)(TMEDA)],DBU,water,0.038384,0.281134,0.453264,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0000,0.0000
48586,48586,70,PPh3,iPrOH,[Ni(oTol)(Cl)(TMEDA)],K3PO4,water,0.038384,0.281134,0.453264,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0.0212,0.1607
48614,48614,70,PPh3,iPrOH,NiCl2.6H2O,DIPEA,MeOH,0.038384,0.281134,0.453264,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5,0.4183,0.8606
48650,48650,70,PPh3,DEC,[Ni(COD)(DQ)],DIPEA,MeOH,0.038384,0.281134,0.453264,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5,0.2750,0.7504


## 2. Converting loaded data to training data and train GP model

In [6]:
# in this dataframe, column 7 onwards, 'ligand_PC1', onwards contains the yields/selectivities and reaction condition features.
descriptor_index = 7 # change depending on your data organisatio. 

train_df = init_x_experiments.iloc[:,descriptor_index:]
train_df.head(5)

,ligand_PC1,ligand_PC2,ligand_PC3,ligand_PC4,ligand_PC5,ligand_PC6,ligand_PC7,ligand_PC8,ligand_PC9,ligand_PC10,...,Cs2CO3,DBU,DIPEA,K3PO4,MeOH,none,water,temperature_descriptor,P_A%,Selectivity_A%
48464,0.038384,0.281134,0.453264,0.4765,0.307554,0.294159,0.332948,0.078107,0.463977,0.289871,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.0000,0.0000
48484,0.038384,0.281134,0.453264,0.4765,0.307554,0.294159,0.332948,0.078107,0.463977,0.289871,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,0.0000,0.0000
48586,0.038384,0.281134,0.453264,0.4765,0.307554,0.294159,0.332948,0.078107,0.463977,0.289871,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0.0212,0.1607
48614,0.038384,0.281134,0.453264,0.4765,0.307554,0.294159,0.332948,0.078107,0.463977,0.289871,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5,0.4183,0.8606
48650,0.038384,0.281134,0.453264,0.4765,0.307554,0.294159,0.332948,0.078107,0.463977,0.289871,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5,0.2750,0.7504


In [7]:
objective_columns = ['P_A%', 'Selectivity_A%']
train_x = train_df.drop(objective_columns, axis=1)
train_y = train_df[objective_columns]
train_x.head(1), train_y.head(5)

(       ligand_PC1  ligand_PC2  ligand_PC3  ligand_PC4  ligand_PC5  ligand_PC6  \
 48464    0.038384    0.281134    0.453264      0.4765    0.307554    0.294159   
 
        ligand_PC7  ligand_PC8  ligand_PC9  ligand_PC10  ...  \
 48464    0.332948    0.078107    0.463977     0.289871  ...   
 
        [Ni(oTol)(Cl)(PPh3)2]  [Ni(oTol)(Cl)(TMEDA)]  Cs2CO3  DBU  DIPEA  \
 48464                    0.0                    0.0     1.0  0.0    0.0   
 
        K3PO4  MeOH  none  water  temperature_descriptor  
 48464    0.0   1.0   0.0    0.0                     0.5  
 
 [1 rows x 53 columns],
          P_A%  Selectivity_A%
 48464  0.0000          0.0000
 48484  0.0000          0.0000
 48586  0.0212          0.1607
 48614  0.4183          0.8606
 48650  0.2750          0.7504)

In [8]:
# convert to torch tensor for input into BO pipeline. 
train_x_tensor = torch.tensor(train_x.to_numpy()).to(**tkwargs)

# get standardised train y from objective columns, inputs are already normalised 
scaler_objectives = StandardScaler()
train_y_std = scaler_objectives.fit_transform(train_y)
train_y_std_tensor = torch.tensor(train_y_std).to(**tkwargs)

## 3. Define reaction condition search space for optimisation

- We define the space of possible reaction conditions we want to explore with BO. Here, we use the originally defined search space dataframe as in `sobol_initialisation.ipynb`, but this can be adjusted depending on experimentalist choice to increase or remove any number of desired experiments. You will have to make your own dataframe feature representation of the experiments you want to run. `3_chemical_space_construction.ipynb` provides a short tutorial, but note that you will have to normalise the search space before use as shown below.

In [9]:
# total possible reaction condition space including reaction descriptors 
total_chem_space = pd.read_csv('../experimental_campaigns/design_spaces/ni_suzuki_chemical_space.csv',index_col=0)
total_chem_descriptors = total_chem_space.iloc[:,descriptor_index:]

# conver the descriptors into torch tensors for model input 
x_space = torch.tensor(total_chem_descriptors.to_numpy()).to(**tkwargs) # total reaction condition space 

# already normalised reaction condition feature space
total_chem_descriptors

,ligand_PC1,ligand_PC2,ligand_PC3,ligand_PC4,ligand_PC5,ligand_PC6,ligand_PC7,ligand_PC8,ligand_PC9,ligand_PC10,...,[Ni(oTol)(Cl)(PPh3)2],[Ni(oTol)(Cl)(TMEDA)],Cs2CO3,DBU,DIPEA,K3PO4,MeOH,none,water,temperature_descriptor
0,0.000000,0.105073,0.319448,0.724348,0.411828,0.366477,0.173392,0.208029,0.898098,0.283199,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.000000,0.105073,0.319448,0.724348,0.411828,0.366477,0.173392,0.208029,0.898098,0.283199,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,0.105073,0.319448,0.724348,0.411828,0.366477,0.173392,0.208029,0.898098,0.283199,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.000000,0.105073,0.319448,0.724348,0.411828,0.366477,0.173392,0.208029,0.898098,0.283199,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.000000,0.105073,0.319448,0.724348,0.411828,0.366477,0.173392,0.208029,0.898098,0.283199,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88795,0.696324,0.354760,0.366684,0.502091,0.358269,0.277274,0.169161,0.032686,0.254501,0.368551,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
88796,0.696324,0.354760,0.366684,0.502091,0.358269,0.277274,0.169161,0.032686,0.254501,0.368551,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
88797,0.696324,0.354760,0.366684,0.502091,0.358269,0.277274,0.169161,0.032686,0.254501,0.368551,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
88798,0.696324,0.354760,0.366684,0.502091,0.358269,0.277274,0.169161,0.032686,0.254501,0.368551,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


## 4. Run Bayesian Optimisation 

Defining the arguments and settings used. If you are intending to run this notebook as a tutorial only, please reduce the `batch_size`, `acqf_samples`, and `max_batch_eval` parameters.

| Arguments | Explanation |
| --- | --- |
| `seed` (int) | Random seed set using pytorch lightning |
| `train_x` (torch.Tensor) | Normalised training tensor of reaction conditions/features |
| `train_y_std` (torch.Tensor) | Standardised training output tensor | 
| `x_space` (torch.Tensor) | Tensor containing all reaction conditions/features desired to be explored/evaluated on by Bayesian Optimisation |
| `batch_size` (int) | Number of experiments to suggest in parallel |
| `kernel` (str) | Kernel hyperparamters for Matern Kernel Gaussian Process. Available choices are `default` and `edboplus` |
| `constrain_strategy` (int) | Strategy to use for constraining (e.g.) the number of temperatures per batch. <br> Choices implemented are `nested` and `naive`. For a full explanation we refer the user to our paper. |
| `n_unique_features` (int) | The number of unique features (temperature in this case) values allowed per batch. |
| `feature_index` (int) | Index of the constrained feature, temperature, in the training input features **tensor**. <br> This depends on your benchmark dataset and must be changed. We have added the feature indexes for our existing datasets |
| `acqf_samples` (int) | Number of samples used to approximate acquisition function. Can be reduced depending on memory constraints. |
| `max_batch_eval` (int) | Controls acquisition function evaluation in batches. Can be reduced depending on memory constraints | 

In [10]:
seed=49

batch_size = 96 # reduce if used for tutorial purposes only
acqf_samples = 1024 # reduce if used for tutorial purposes only
max_batch_eval = 512 # reduce if used for tutorial purposes only
kernel = 'edboplus' # choose GP kernel hyperparameters from edboplus
constrain_strategy = 'nested'
n_unique_features = 2
feature_index = -1 # in dataframes above, temperature_descriptor is the last column of x_space and train_x dataframes. 

In [11]:
Campaign = BayesianOptimisation(device=tkwargs,seed=seed)

new_suggestions = Campaign.run_optimisation_iteration(
    train_x=train_x_tensor,
    train_y_std=train_y_std_tensor,
    x_space=x_space,
    batch_size=batch_size,
    kernel=kernel,
    acqf_samples=acqf_samples,
    max_batch_eval=max_batch_eval,
    constrain_strategy='nested',
    n_unique_features=2,
    feature_index=-1,
    )

2024-08-13 11:48:33,592 - minerva.bayesopt - INFO - Initialising BayesianOptimisation with seed 49
Seed set to 49
Optimizing batch: 100%|██████████| 96/96 [03:25<00:00,  2.14s/it]


In [12]:
new_index = get_index_from_lookup(new_suggestions, x_space) 
new_experiments = total_chem_space.iloc[new_index]
new_experiments

,rxn_id,temperature,ligand,solvent,precursor,base,cosolvent,ligand_PC1,ligand_PC2,ligand_PC3,...,[Ni(oTol)(Cl)(PPh3)2],[Ni(oTol)(Cl)(TMEDA)],Cs2CO3,DBU,DIPEA,K3PO4,MeOH,none,water,temperature_descriptor
86678,86678,100,cataCXium PiPr,DMI,NiCl2.6H2O,DIPEA,MeOH,0.355281,0.468712,0.078582,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
80918,80918,100,P(2-OMe-C6H4)3,DMI,NiCl2.6H2O,DIPEA,MeOH,0.318098,0.246090,0.586817,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
51878,51878,70,P(2-OMe-C6H4)3,DMI,NiCl2.6H2O,DIPEA,MeOH,0.318098,0.246090,0.586817,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
80870,80870,100,P(2-OMe-C6H4)3,tAmOH,NiCl2.6H2O,DIPEA,MeOH,0.318098,0.246090,0.586817,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
64598,64598,70,cataCXium PiPr,iPrOH,NiCl2.6H2O,DIPEA,MeOH,0.355281,0.468712,0.078582,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51698,51698,70,P(2-OMe-C6H4)3,EtOAc,[Ni(oTol)(Cl)(TMEDA)],DIPEA,MeOH,0.318098,0.246090,0.586817,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
64766,64766,70,cataCXium PiPr,tAmOH,[Ni(oTol)(Cl)(PPh3)2],DIPEA,MeOH,0.355281,0.468712,0.078582,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
63134,63134,70,PhCPhos,MeTHF,[Ni(oTol)(Cl)(PPh3)2],DIPEA,MeOH,0.608197,0.372141,0.368333,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
63494,63494,70,PhCPhos,tAmOH,NiCl2.6H2O,DIPEA,MeOH,0.608197,0.372141,0.368333,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.5
